# Projeto Final de Spark - Avançado

## Campanha Nacional de Vacinação contra Covid-19

Aluno: Marcelo de Oliveira Santos

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

## Nível Avançado:

Replicar as visualizações do site “https://covid.saude.gov.br/”, porém acessando diretamente a API de Elastic.

Link oficial para todas as informações:  https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao

Informações para se conectar ao cluster:

    > URL https://imunizacao-es.saude.gov.br/desc-imunizacao
    > Nome do índice: desc-imunizacao
    > Credenciais de acesso
    > Usuário: imunizacao_public
    > Senha: qlto5t&7r_@+#Tlstigi

In [1]:
import requests
import pandas as pd
import time

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
#Acesso inicial para gravar '_scroll_id'
covid_API = requests.get('https://imunizacao-es.saude.gov.br/_search?scroll=1m',
                         auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                         params={"size":10000})

In [3]:
covid_API.json()['hits']['total']

{'value': 116310688, 'relation': 'eq'}

In [4]:
#'_scroll_id' = id temporário que será utilizado para navegar nos scrolls com 10000 registros cada
covid_API.json()['_scroll_id']

'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoAxZNS1hZTVE5c1RFV1d5TnBQdV94RHVnAAAAAG2Bx-EWVGNTY2RHVmdRTUtxVjBiQXFFMEJCURZlR0trMDNRRlFreTlFRWtxel9vME1BAAAAAGR9LH8WeUtZa0E0b1ZTZlNjam5veUxfS1ZHdxY3ZGlmdS1PcFNXbTBxTUhIbWhpdnB3AAAAAGu43uIWU1BFYmVBMDBRV1dULWt4VDJxczEwUQ=='

In [5]:
#Número de registros por Scroll
len(covid_API.json()['hits']['hits'])

10000

In [6]:
#Scroll = 0 

count_scroll = 0

scroll_id = covid_API.json()['_scroll_id']
covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll',
                                auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                                params={"scroll_id":scroll_id, "scroll":'1m'})

#recebe infomações HITS, seleciona dict _source
df_covid = pd.DataFrame([x['_source'] for x in covid_API_scroll.json()['hits']['hits']])

In [7]:
df_covid.head(2)

,estabelecimento_razaoSocial,vacina_dataAplicacao,vacina_grupoAtendimento_codigo,estabelecimento_valor,@timestamp,sistema_origem,vacina_lote,id_sistema_origem,estalecimento_noFantasia,paciente_endereco_coIbgeMunicipio,...,vacina_descricao_dose,vacina_fabricante_nome,vacina_categoria_codigo,paciente_endereco_uf,vacina_categoria_nome,redshift,vacina_nome,paciente_racaCor_valor,paciente_id,paciente_enumSexoBiologico
0,PREFEITURA MUNICIPAL DE SANTO ANDRE,2021-02-16T00:00:00.000Z,000926,0008699,2021-07-09T15:56:02.556Z,VACIVIDA,202010029,18262,UBS JARDIM IRENE,355030,...,2ª Dose,Sinovac,9,SP,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,SEM INFORMACAO,464ace8c056e3eb67ebdc8ad0a198b74f259d11700003b...,F
1,MUNICIPIO DE ITAJAI,2021-02-08T00:00:00.000Z,000907,2513870,2021-07-09T15:56:02.557Z,G-MUS,4120Z005,17901,CRESCEM CENTRO DE REFERENCIA DA SAUDE DA CRIAN...,420910,...,1ª Dose,FUNDACAO OSWALDO CRUZ,9,SC,Trabalhadores de Saúde,v2,Vacina Covid-19 - Covishield,BRANCA,2890fdfe35dd36eac48539c3740463bb4862d92e3f86cb...,M


In [8]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 37 columns):
estabelecimento_razaoSocial                 10000 non-null object
vacina_dataAplicacao                        10000 non-null object
vacina_grupoAtendimento_codigo              10000 non-null object
estabelecimento_valor                       10000 non-null object
@timestamp                                  10000 non-null object
sistema_origem                              10000 non-null object
vacina_lote                                 10000 non-null object
id_sistema_origem                           10000 non-null object
estalecimento_noFantasia                    10000 non-null object
paciente_endereco_coIbgeMunicipio           10000 non-null object
paciente_endereco_coPais                    10000 non-null object
estabelecimento_uf                          10000 non-null object
paciente_nacionalidade_enumNacionalidade    9986 non-null object
paciente_endereco_nmPais           

In [9]:
%%time
#Loop para repetir processo anterior até hits  = []

"""
Para zerar hits preciso otimizar o loop abaixo
~2s por scroll, sendo 11.631 scroll
"""

while count_scroll < 10:
#while len(covid_API_scroll.json()['hits']['hits']) != 0:
    scroll_id = covid_API.json()['_scroll_id']
    covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll',
                                    auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                                    params={"scroll_id":scroll_id, "scroll":'1m'})

    #covid_API_scroll.json()
    #covid_API_scroll.json()['hits']['hits']
    
    df_covid = df_covid.append([x['_source'] for x in covid_API_scroll.json()['hits']['hits']])
    count_scroll += 1

CPU times: user 9.25 s, sys: 360 ms, total: 9.61 s
Wall time: 18 s


In [10]:
%%time
covid_vac = spark.createDataFrame(df_covid)

CPU times: user 33.6 s, sys: 36 ms, total: 33.7 s
Wall time: 35.1 s


In [11]:
covid_vac.count()

110000

In [14]:
covid_vac.groupBy("paciente_endereco_uf")\
        .agg(countDistinct("paciente_id").alias("vacinados"),\
            ).sort(asc("paciente_endereco_uf")).show(5)

+--------------------+---------+
|paciente_endereco_uf|vacinados|
+--------------------+---------+
|                    |      550|
|                  AC|      411|
|                  AL|     1283|
|                  AM|     2513|
|                  AP|      269|
+--------------------+---------+
only showing top 5 rows



In [16]:
covid_vac.groupBy("vacina_descricao_dose")\
        .agg(countDistinct("paciente_id").alias("vacinados"),\
            ).sort(asc("vacina_descricao_dose")).show(5)

+---------------------+---------+
|vacina_descricao_dose|vacinados|
+---------------------+---------+
|              1ª Dose|    66611|
|              2ª Dose|    43368|
+---------------------+---------+



In [17]:
covid_vac.groupBy("vacina_dataAplicacao")\
        .agg(countDistinct("paciente_id").alias("vacinados"),\
            ).sort(desc("vacina_dataAplicacao")).show(5)

+--------------------+---------+
|vacina_dataAplicacao|vacinados|
+--------------------+---------+
|2021-05-04T00:00:...|     2672|
|2021-05-03T00:00:...|    23546|
|2021-05-02T00:00:...|     4089|
|2021-05-01T00:00:...|     6566|
|2021-04-30T00:00:...|     7575|
+--------------------+---------+
only showing top 5 rows



In [18]:
covid_vac.groupBy("vacina_nome")\
        .agg(count("paciente_id").alias("vacinados"),\
            ).sort(desc("vacina_nome")).show(5)

+--------------------+---------+
|         vacina_nome|vacinados|
+--------------------+---------+
|Vacina covid-19 -...|        8|
|Vacina Covid-19 -...|    54175|
|Covid-19-Coronava...|    54368|
|Covid-19-AstraZeneca|     1449|
+--------------------+---------+



##### Próximos passos:

    > conseguir subir todos os hits;